<h1 align = 'center'> Coleridge Initiative - Show US the Data </h1>
<h2 align = 'center'>  Group </h2>

## 1. Setup

In [1]:
#from bs4 import BeautifulSoup
import zipfile
import os
import json
import numpy as np 
import pandas as pd 

os.chdir('src')
import vocab
os.chdir('..')

DATA_FOLDER = "Data"
TRAIN_FOLDER = "Data/train/"

## 2. Data

In [ ]:
def get_data(data_folder):
    '''
    Download data from kaggle and save to /Data/
    '''
    
    #Create data folder
    try: 
        os.mkdir(data_folder)
        print('Data Folder Created')
    except:
        print('Data Folder Already Created')

    #Download data from kaggle and unzip.
    os.chdir(data_folder)
    !kaggle competitions download -c coleridgeinitiative-show-us-the-data
    with zipfile.ZipFile("coleridgeinitiative-show-us-the-data.zip","r") as zip_ref:
        zip_ref.extractall(data_folder + '/')
    os.chdir('..')

#get_data(DATA_FOLDER)

In [2]:
train_df = pd.read_csv(DATA_FOLDER + '/train.csv')
train_df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


In [4]:
train_files = os.listdir(TRAIN_FOLDER)

In [7]:
train_df[train_df.Id == train_files[0][:-5]]

,Id,pub_title,dataset_title,dataset_label,cleaned_label
1582,0007f880-0a9b-492d-9a58-76eb0b0e0bd7,The Impact of ICT Training on Income Generatio...,Program for the International Assessment of Ad...,Program for the International Assessment of Ad...,program for the international assessment of ad...


In [26]:
appearance = {}
found = 0
for fname in train_files: 
    with open(TRAIN_FOLDER + fname) as f:
            sample_data = json.loads(f.read())
            label = train_df[train_df.Id == fname[:-5]].cleaned_label.iloc[0]
            for sec in sample_data:
                text = sec['text'].lower()
                if label in text:
                    try:
                        appearance[sec['section_title'].lower().strip()] += 1
                    except:
                        appearance[sec['section_title'].lower().strip()] = 0
                        found += 1

In [27]:
found / len(train_files)

0.8166387259010897

In [28]:
(pd.Series(appearance)/ len(train_files)).sort_values(ascending =False).head(10)

introduction    0.159961
abstract        0.096326
discussion      0.090319
                0.073903
data            0.030246
results         0.022073
methods         0.020327
participants    0.018092
conclusion      0.016206
subjects        0.015647
dtype: float64

## 3. Preliminary EDA

In [31]:
print("Total number of files for training data: {}".format(str(len(train_files))))

Total number of files for training data: 14316


In [45]:
section_titles = []
sections_counts = {}
for fname in train_files: 
    with open(TRAIN_FOLDER + fname) as f:
        sample_data = json.loads(f.read())
    section_title = []
    for i in sample_data:
        section_title.append(i['section_title'])
    section_titles.append(section_title)
    for sec in section_title:
        try:
            sections_counts[sec] += 1
        except:
            sections_counts[sec] = 0        

In [56]:
pd.Series(sections_counts).sort_values(ascending = False).head(20)

Abstract                 8289
Introduction             6616
Results                  4862
Discussion               4650
                         4273
Methods                  2411
Conclusion               2274
Conclusions              2125
INTRODUCTION             1999
RESULTS                  1523
DISCUSSION               1442
Data                      880
Participants              813
METHODS                   733
Background                731
Method                    675
Statistical analysis      625
CONCLUSION                596
Materials and Methods     580
Limitations               576
dtype: int64

In [60]:
train_df.dataset_title.value_counts().head(10)

Alzheimer's Disease Neuroimaging Initiative (ADNI)       6144
Baltimore Longitudinal Study of Aging (BLSA)             1589
Trends in International Mathematics and Science Study    1163
Early Childhood Longitudinal Study                       1011
SARS-CoV-2 genome sequence                                860
Census of Agriculture                                     743
Education Longitudinal Study                              676
Agricultural Resource Management Survey                   660
North American Breeding Bird Survey (BBS)                 585
National Education Longitudinal Study                     550
Name: dataset_title, dtype: int64

In [61]:
train_df.dataset_label.value_counts().head(10)

ADNI                                                     3673
Alzheimer's Disease Neuroimaging Initiative (ADNI)       2400
Trends in International Mathematics and Science Study    1163
Baltimore Longitudinal Study of Aging                    1156
Early Childhood Longitudinal Study                       1011
Education Longitudinal Study                              676
Census of Agriculture                                     643
Agricultural Resource Management Survey                   623
National Education Longitudinal Study                     550
Rural-Urban Continuum Codes                               490
Name: dataset_label, dtype: int64

In [65]:
train_df.cleaned_label.value_counts().head(10)

adni                                                     3673
alzheimer s disease neuroimaging initiative adni         2400
trends in international mathematics and science study    1163
baltimore longitudinal study of aging                    1156
early childhood longitudinal study                       1011
education longitudinal study                              676
census of agriculture                                     643
agricultural resource management survey                   623
national education longitudinal study                     550
rural urban continuum codes                               490
Name: cleaned_label, dtype: int64

In [74]:
train_df.describe()

,Id,pub_title,dataset_title,dataset_label,cleaned_label
count,19661,19661,19661,19661,19661
unique,14316,14271,45,130,130
top,170113f9-399c-489e-ab53-2faf5c64c5bc,Science and Engineering Indicators 2014,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni
freq,22,22,6144,3673,3673


## 4. Dataset Preparation

In [6]:
vocab.load_vocab(os.getcwd() +'/' + TRAIN_FOLDER, 1)

Using the saved vocab.
